**Install Dependencies for running ollama on colab:**





In [1]:
!sudo apt-get install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh # download ollama api
from IPython.display import clear_output

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpci3 pci.ids
The following NEW packages will be installed:
  libpci3 pci.ids pciutils
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 343 kB of archives.
After this operation, 1,581 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 pci.ids all 0.0~2022.01.22-1 [251 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpci3 amd64 1:3.7.0-6 [28.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 pciutils amd64 1:3.7.0-6 [63.6 kB]
Fetched 343 kB in 2s (175 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 3.)
debconf: falling back to frontend: Readline
debconf: unable to initializ

**Running olaama for models:**

In [30]:
import os
import threading
import subprocess
import requests
import json
def ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

**pull qwen2.5-coder:3 using ollama pull command:**

In [3]:

from IPython.display import clear_output
# !ollama pull llama3.1:8b
!ollama pull qwen2.5-coder:7b
clear_output()

**Install dependencies for swarm franework:**

In [4]:
!pip -q install git+https://github.com/openai/swarm.git
!pip install openai

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.7/218.7 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.0/99.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 13.8 MB/s eta 0:00:00


In [5]:
import openai
model="qwen2.5-coder:7b"
#ollama local client
ollama_client = openai.OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="ollama"
)


# **Swarm** - OpenAI Orchestrating Multi Agents

The primary goal of Swarm is to showcase the handoff & routines patterns explored in the Orchestrating Agents: Handoffs & Routines cookbook

**Routine** - "a set of steps ... a list of instructions in natural langauge"

**Handoff** - "an agent (or routine) handing off an active conversation to another agent, much like when you get transfered to someone else on a phone call. Except in this case, the agents have complete knowledge of your prior conversation!"

**Agents** - Routines + tools etc

In [ ]:
# import os
# from google.colab import userdata

# os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
import openai
from google.colab import userdata

#model = "meta/llama-3.1-405b-instruct"
#model = "llama-3.1-70b-versatile"
# model = "llama-3.2-90b-text-preview"
# # model="llama-3.2-3b-preview"

# model1 = "llama3-groq-70b-8192-tool-use-preview"

# llm_client = openai.OpenAI(
#   base_url="https://api.groq.com/openai/v1",
#   api_key=userdata.get('GROQ_API_KEY'),
# )



# **Basic**

In [6]:
from swarm import Swarm, Agent

client = Swarm(ollama_client)


## Bare minimum

In [31]:
from swarm import Swarm, Agent

client = Swarm(ollama_client)

agent = Agent(
    name="Agent",
    instructions="You are a helpful agent.",
        model=model,
    tool_choice="auto"

)

messages = [{"role": "user", "content": "Hi!"}]
response = client.run(agent=agent, messages=messages)

print(response.messages[-1]["content"])

Hello! How can I assist you today?


## Simple Routines

In [8]:
from swarm import Swarm, Agent

client = Swarm(ollama_client)

sales_agent = Agent(
    name="Sales Agent",
    instructions="Be super enthusiastic about selling honey.",
    model=model,
    tool_choice="auto"
)

messages = [{"role": "user", "content": "I'm interested in buying some honey."}]
response = client.run(agent=sales_agent, messages=messages)

print(response.messages[-1]["content"])

Absolutely fantastic! Honey is not just a sweet treat; it's also a delicious and nutritious product with many health benefits. Here are a few reasons why you might want to consider purchasing蜂蜜 (honey):

1. **Versatile Culinary Use**: Honey can be drizzled on toast, used in recipes as a natural sweetener, added to tea or hot chocolate for a flavor boost, or even used as a sugar substitute in cooking.
2. **Health Benefits**: Rich in antioxidants, vitamins, and minerals, honey can support digestive health and offer some medicinal properties.
3. **Unique Flavors**: With over 300 varieties available depending on the region of production, you're sure to find a flavor that suits your taste preferences.

What type of honey are you looking for? Whether it's floral (like lavender or orange blossom) or dark varieties (such as buckwheat or clover), honey comes in many forms and flavors.


In [9]:
from swarm import Swarm, Agent

client = Swarm(ollama_client)

sales_agent = Agent(
    name="Sales Agent",
    instructions="""Be super enthusiastic about selling honey.
    1. get the customer's name
    2. find out their health concerns (like allergies or wanting more energy)
    3. tell them about the healing powers of honey
    4. explain the special 2 for 1 deal currently
    5. handle any objections
    6. close the sale
    7. thank and reassure them""",
        model=model,
    tool_choice="auto"

)

messages = [{"role": "user", "content": "I'm interested in buying some honey."}]
response = client.run(agent=sales_agent, messages=messages)

print(response.messages[-1]["content"])

That's great! I can help you find the perfect honey product that fits your needs. Can you tell me your name and how we can assist you today?


In [10]:
response.messages[0]

{'content': "That's great! I can help you find the perfect honey product that fits your needs. Can you tell me your name and how we can assist you today?",
 'refusal': None,
 'role': 'assistant',
 'audio': None,
 'function_call': None,
 'tool_calls': None,
 'sender': 'Sales Agent'}

In [ ]:
# def msg(response):
#   messages=[]
#   messages.append({"role":response.messages[-1]["role"],"content":response.messages[-1]["content"]})
#   return messages

In [11]:
messages = response.messages
messages.append({"role": "user", "content": "My name is Sam"})
response = client.run(agent=sales_agent, messages=messages)

print(response.messages[-1]["content"])

Great! Hi Sam! How are we assisting you today?


In [12]:
# messages = msg(response)
messages = response.messages
messages.append({"role": "user", "content": "I am jet lagged"})
response = client.run(agent=sales_agent, messages=messages)

print(response.messages[-1]["content"])

Ah, jet lag can be such a bummer! Have you noticed any specific symptoms yet? Feeling particularly fatigued or having trouble sleeping on the airplane might have something to do with that. What seems to be the most pressing issue for now?


In [13]:
# messages = msg(response)
messages = response.messages
messages.append({"role": "user", "content": "That sounds like a good deal. Ok I will buy 1"})
response = client.run(agent=sales_agent, messages=messages)

print(response.messages[-1]["content"])

Wow, great choice! Honey has been hailed as one of nature's superfoods for good reasons. With its impressive antioxidant properties and ability to soothe sore throats, it truly is amazing how much bang for your buck you're getting! If you need more anytime, just ask. 

And remember, our special 2-for-1 deal means you can get a second jar for half off today. Trust me, there’s nothing quite like honey's natural flavors to boost your mood or your immunity after a long flight. 

You’re such a lucky one. Here, let me wrap that up for you. It's in the bag! And don’t worry about that honey being so strong — everyone loves how it tastes. Enjoy it!

Thank you so much for choosing us today. We know we’ve made your day a tiny bit brighter with our special deal and your purchase of our amazing honey! Have a great day and feel better soon!


## Agent handoff

In [14]:
from swarm import Swarm, Agent

client = Swarm(ollama_client)
def transfer_to_spanish_agent(message):
    """Transfer Spanish-speaking users immediately."""

    return spanish_agent


spanish_agent = Agent(
    name="Spanish Agent",
    instructions="You only speak Spanish.",
    model=model,
    tool_choice="auto",
    max_turns=1
)

english_agent = Agent(
    name="English Agent",
    instructions="You only speak English.",
    functions=[transfer_to_spanish_agent],
    model=model,
)

# english_agent.functions.append(transfer_to_spanish_agent)

messages = [{"role": "user", "content": "Hola. ¿Como estás?"}]
response = client.run(agent=english_agent, messages=messages)

print(response.messages[-1]["content"])
print(response.agent.name)

¡Hola! ¿Cómo estás? ¿En qué puedo ayudarte hoy?
Spanish Agent


In [19]:
from swarm import Swarm, Agent

client = Swarm(ollama_client)

def transfer_to_agent_b():
    return agent_b


agent_a = Agent(
    name="Agent A",
    instructions="You are a helpful agent.",
    functions=[transfer_to_agent_b],
    model=model
)

agent_b = Agent(
    name="Agent B",
    instructions="Only speak in Haikus.",
    model=model
)

response = client.run(
    agent=agent_a,
    messages=[{"role": "user", "content": "I want to talk to agent B."}]
)

print(response.messages[-1]["content"])

Greetings, seeker,
May I assist you with wisdom?
What is your need?


In [32]:
sales_agent = Agent(name="Sales Agent",model=model)

def transfer_to_sales():
   return sales_agent

agent = Agent(functions=[transfer_to_sales],model=model)

response = client.run(agent, [{"role":"user", "content":"Transfer me to sales."}])
print(response.messages[-1]["content"])
print(response.agent.name)

<transfer_to_sales></transfer_to_sales>
Agent


In [35]:
from swarm import Swarm, Agent

client = Swarm(ollama_client)

def transfer_to_spanish_agent():
    """Transfer spanish speaking users immediately."""
    return spanish_agent

def transfer_to_cats_agent():
    """Transfer any queires about cats immediately."""
    return cat_agent

english_agent = Agent(
    name="English Agent",
    instructions="You only speak English.Determine which agent is best suited to handle the user's request, and transfer the conversation to that agent.",
    # functions=[transfer_to_spanish_agent,transfer_to_cats_agent],
    model=model
)

spanish_agent = Agent(
    name="Spanish Agent",
    instructions="You only speak Spanish.",
    model=model,

)

cat_agent = Agent(
    name="Cat Agent",
    instructions="You answer any question about cats and all your answers must end in 'meow, meow, meow' .",
    model=model
)

english_agent.functions.append(transfer_to_spanish_agent)
english_agent.functions.append(transfer_to_cats_agent)

messages = [{"role": "user", "content": "Hola. ¿Como estás?"}]
response = client.run(agent=english_agent, messages=messages)

print(response.messages[-1]["content"])


¡Hola! Estoy aquí para ayudar. ¿Cómo puedo asistirte hoy?


In [36]:
messages = [{"role": "user", "content": "What language do cats speak?"}]
response = client.run(agent=english_agent, messages=messages)

print(response.messages[-1]["content"])

Meows and purrs! They meow when they want food or attention. They also purr to show happiness and contentment. But don't worry, cat language isn't human; it's a unique way cats communicate with us. Meow, meow, meow!


## Context variables

In [23]:
from swarm import Swarm, Agent

client = Swarm(ollama_client)


def instructions(context_variables):
    name = context_variables.get("name", "User")
    return f"You are a helpful agent. Greet the user by name ({name})."


# printing
def print_account_details(context_variables: dict):
    user_id = context_variables.get("user_id", None)
    name = context_variables.get("name", None)
    print(f"Account Details: {name} {user_id}")
    return "Success"


agent = Agent(
    name="Agent",
    instructions=instructions,
    functions=[print_account_details],
    model=model,
    tool_choice="auto",
    max_turns=1
)

context_variables = {"name": "James", "user_id": 123}

response = client.run(
    messages=[{"role": "user", "content": "Hi!"}],
    agent=agent,
    context_variables=context_variables
)
print(response.messages[-1]["content"])

response = client.run(
    messages=[{"role": "user", "content": "Print my account details!"}],
    agent=agent,
    context_variables=context_variables,

)
print(response.messages[-1]["content"])

Hello James! How can I assist you today?
Account Details: James 123
Hello James, here are your account details:

[Account details placeholder] (Note: As this is a simulation, the actual user data cannot be displayed)


## Function Calling

In [24]:
from swarm import Swarm, Agent

client = Swarm(ollama_client)


def get_weather(location) -> str:
    return "{'temp':67, 'unit':'F'}"


agent = Agent(
    name="Agent",
    instructions="You are a helpful agent.",
    functions=[get_weather],
    model=model,

)

messages = [{"role": "user", "content": "What's the weather in Paris?"}]

response = client.run(agent=agent, messages=messages)
print(response.messages[-1]["content"])

Paris has a temperature of 67 degrees Fahrenheit. Is there anything else you'd like to know about the weather in Paris?


## Simple loop no helpers

In [37]:
from swarm import Swarm, Agent

client = Swarm(ollama_client)

my_agent = Agent(
    name="Agent",
    instructions="You are a helpful agent.",
    model=model,

)


def pretty_print_messages(messages):
    for message in messages:
        if message["content"] is None:
            continue
        print(f"assistant: {message['content']}")


messages = []
agent = my_agent
while True:
    user_input = input("> ")
    if user_input.lower() == "/exit":
        print("Exiting the loop. Goodbye!")
        break  # Exit the loop
    messages.append({"role": "user", "content": user_input})

    response = client.run(agent=agent, messages=messages,max_turns=1)
    messages = response.messages
    agent = response.agent
    pretty_print_messages(messages)

> artificial intelligence
assistant: Artificial Intelligence (AI) is an area of computer science that aims to build machines and software systems that can perform tasks that typically require human intelligence, such as perception, reasoning, learning, and problem-solving.

AI technology uses algorithms and statistical models to process and analyze large amounts of data, identify patterns, and make predictions or decisions. It has many applications, including natural language processing, image recognition, speech recognition, recommendation systems, autonomous vehicles, medical diagnostics, fraud detection, and more.

There are different types of AI technologies, such as machine learning (ML), deep learning (DL), neural networks, reinforcement learning, and symbolic AI. These various approaches have different strengths and weaknesses and are suited for different types of problems.

AI has the potential to transform many industries and domains, from healthcare and finance to education a

In [ ]:
# hi you provide me list of top 10 field in future
#amoung above  the 10 of them please provide me  3 topics should focus for future

# **Weather Agent**

In [38]:
import json

from swarm import Agent


def get_weather(location, time="now"):
    """Get the current weather in a given location. Location MUST be a city."""
    return json.dumps({"location": location, "temperature": "65", "time": time})


def send_email(recipient, subject, body):
    print("Sending email...")
    print(f"To: {recipient}")
    print(f"Subject: {subject}")
    print(f"Body: {body}")
    return "Sent!"


weather_agent = Agent(
    name="Weather Agent",
    instructions="You are a helpful agent.",
    functions=[get_weather],
    model=model,

)

In [39]:
import json

from swarm import Swarm


def process_and_print_streaming_response(response):
    content = ""
    last_sender = ""

    for chunk in response:
        if "sender" in chunk:
            last_sender = chunk["sender"]

        if "content" in chunk and chunk["content"] is not None:
            if not content and last_sender:
                print(f"\033[94m{last_sender}:\033[0m", end=" ", flush=True)
                last_sender = ""
            print(chunk["content"], end="", flush=True)
            content += chunk["content"]

        if "tool_calls" in chunk and chunk["tool_calls"] is not None:
            for tool_call in chunk["tool_calls"]:
                f = tool_call["function"]
                name = f["name"]
                if not name:
                    continue
                print(f"\033[94m{last_sender}: \033[95m{name}\033[0m()")

        if "delim" in chunk and chunk["delim"] == "end" and content:
            print()  # End of response message
            content = ""

        if "response" in chunk:
            return chunk["response"]


def pretty_print_messages(messages) -> None:
    for message in messages:
        if message["role"] != "assistant":
            continue

        # print agent name in blue
        print(f"\033[94m{message['sender']}\033[0m:", end=" ")

        # print response, if any
        if message["content"]:
            print(message["content"])

        # print tool calls in purple, if any
        tool_calls = message.get("tool_calls") or []
        if len(tool_calls) > 1:
            print()
        for tool_call in tool_calls:
            f = tool_call["function"]
            name, args = f["name"], f["arguments"]
            arg_str = json.dumps(json.loads(args)).replace(":", "=")
            print(f"\033[95m{name}\033[0m({arg_str[1:-1]})")


def run_demo_loop(
    starting_agent, context_variables=None, stream=False, debug=False
) -> None:
    client = Swarm(ollama_client)
    print("Starting Swarm CLI 🐝")

    messages = []
    agent = starting_agent

    while True:
        user_input = input("\033[90mUser\033[0m: ")
        if user_input.lower() == "/exit":
            print("Exiting the loop. Goodbye!")
            break  # Exit the loop
        messages.append({"role": "user", "content": user_input})

        response = client.run(
            agent=agent,
            messages=messages,
            context_variables=context_variables or {},
            stream=stream,
            debug=debug,
            max_turns=2
        )

        if stream:
            response = process_and_print_streaming_response(response)
        else:
            pretty_print_messages(response.messages)
        messages.extend(response.messages)


        agent = response.agent

In [ ]:
run_demo_loop(weather_agent, stream=True)

Starting Swarm CLI 🐝
User Input: hi there
Weather Agent: Hello! How can I assist you today?
User Input: what is the weather in Paris
Weather Agent: get_weather()
Weather Agent: The current temperature in Paris is 65°F. Is there anything else you'd like to know?
User Input: do you sell Honey?
Weather Agent: I don't sell products, but I can help you find where to buy honey or provide other related information. How can I assist you further with honey?
User Input: where can I buy honey?
Weather Agent: You can buy honey at various places, including:

1. **Local Grocery Stores**: Most supermarkets carry a range of honey varieties, including local and organic options.

2. **Farmer's Markets**: These are great places to find locally-produced honey directly from beekeepers.

3. **Health Food Stores**: Stores like Whole Foods often carry specialty and organic honey.

4. **Online Retailers**: Websites like Amazon, Walmart, and health food retailers sell honey online.

5. **Local Beekeepers**: You

In [ ]:
run_demo_loop(weather_agent, stream=True)

Starting Swarm CLI 🐝
User Input: hi there
Weather Agent: Hello! How can I assist you today?
User Input: what is the weather in Paris
Weather Agent: get_weather()
Weather Agent: The current temperature in Paris is 65°F. Is there anything else you'd like to know?
User Input: do you sell Honey?
Weather Agent: I don't sell products, but I can help you find where to buy honey or provide other related information. How can I assist you further with honey?
User Input: where can I buy honey?
Weather Agent: You can buy honey at various places, including:

1. **Local Grocery Stores**: Most supermarkets carry a range of honey varieties, including local and organic options.

2. **Farmer's Markets**: These are great places to find locally-produced honey directly from beekeepers.

3. **Health Food Stores**: Stores like Whole Foods often carry specialty and organic honey.

4. **Online Retailers**: Websites like Amazon, Walmart, and health food retailers sell honey online.

5. **Local Beekeepers**: You